In [1]:
!pip install grpcio grpcio-tools utils
!pip list | grep -e grpcio -e protobuf
!pip install grpcio==1.56.0 grpcio-tools==1.33.2 protobuf==3.20.3
!pip list | grep -e grpcio -e protobuf


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
grpcio                          1.56.0
grpcio-tools                    1.33.2
protobuf                        3.20.3

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
grpcio                          1.56.0
grpcio-tools                    1.33.2
protobuf                        3.20.3

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
grpc_host = 'modelmesh-serving'
grpc_port = 8033
model_name = 'spleenv4'
unet_model_name = 'spleenv4'

In [3]:
import sys
sys.path.append('./utils')

# grpc_predict_v2_pb2 and grpc_predict_v2_pb2_grpc were created from grpc_predict_v2.proto using protoc
import grpc
import utils.grpc_predict_v2_pb2 as grpc_predict_v2_pb2
import utils.grpc_predict_v2_pb2_grpc as grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
response

name: "spleenv4__isvc-9f5498d4d3"
versions: "1"
platform: "onnxruntime_onnx"
inputs {
  name: "modelInput"
  datatype: "FP32"
  shape: -1
  shape: 1
  shape: 96
  shape: 96
  shape: 96
}
outputs {
  name: "modelOutput"
  datatype: "FP32"
  shape: -1
  shape: 2
  shape: 96
  shape: 96
  shape: 96
}

In [4]:
import grpc
import grpc_predict_v2_pb2
import grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

# request = grpc_predict_v2_pb2.ModelMetadataRequest(name=textencoder_model_name)
# response = stub.ModelMetadata(request)
# print(response)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=unet_model_name)
response = stub.ModelMetadata(request)
print(response)

name: "spleenv4__isvc-9f5498d4d3"
versions: "1"
platform: "onnxruntime_onnx"
inputs {
  name: "modelInput"
  datatype: "FP32"
  shape: -1
  shape: 1
  shape: 96
  shape: 96
  shape: 96
}
outputs {
  name: "modelOutput"
  datatype: "FP32"
  shape: -1
  shape: 2
  shape: 96
  shape: 96
  shape: 96
}



In [4]:
import numpy as np
import grpc
import utils.grpc_predict_v2_pb2 as grpc_predict_v2_pb2
import utils.grpc_predict_v2_pb2_grpc as grpc_predict_v2_pb2_grpc

# gRPC host and port
grpc_host = 'modelmesh-serving'
grpc_port = 8033

# Model name
model_name = 'spleenv4'

# Create gRPC channel
channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

# Sample data
sample_data = np.random.rand(1, 96, 96, 96).astype(np.float32)  # Generate random sample data

def grpc_request(data):
    # Create input tensor
    input_tensor = grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor()
    input_tensor.name = "modelInput"
    input_tensor.datatype = "FP32"
    input_tensor.shape.extend([-1, 1, 96, 96, 96])
    input_tensor.contents.fp32_contents.extend(data.flatten().tolist())  # Flatten the data

    # Create request
    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = model_name
    request.inputs.extend([input_tensor])

    # Make gRPC call
    response = stub.ModelInfer(request)
    
    # Parse response
    result_arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
    
    return result_arr

# Split the sample data into smaller batches
batch_size = 4
num_batches = sample_data.shape[0] // batch_size
predictions = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_data = sample_data[start_idx:end_idx]
    prediction = grpc_request(batch_data)
    predictions.append(prediction)

# Concatenate predictions from all batches if predictions were made
final_prediction = np.concatenate(predictions) if predictions else np.array([])

# Print the final prediction
print("Final Prediction:", final_prediction)


Final Prediction: []
